

# Primeros Pasos: Configuración del Entorno de Base de Datos con Docker y MongoDB Compass

Para construir los cimientos de nuestra aplicación, hemos configurado un entorno de base de datos robusto utilizando `Docker` para la gestión de contenedores y `MongoDB Compass` como nuestra interfaz gráfica para interactuar con la base de datos.

1. Despliegue de la Base de Datos MongoDB con Docker

```sh
docker run --name tuTiendaDB --detach --mount src=dbAppTienda,dst=/data/db --publish 27017:27017 mongo:latest
```

`--publish 27017:27017:` El primer 27017 se refiere al puerto de la máquina HOST, en mi caso mi WSL; El segundo 27017 se refiere al puerto interno del contenedor donde MongoDB está escuchando las conexiones. Sin este, no sera posible intercatur desde `MongoDB Compass`

2. Acceso al Contenedor para Interacción Directa (Shell)

```sh
docker exec -it tuTiendaDB bash
```

Una vez dentro del SHELL del contenedor, usar el comando `mongosh`, que es la nueva interfaz de línea de comandos para MongoDB. 

3. Conexión a la Base de Datos con MongoDB Compass.

`MongoDB Compass` es nuestra herramienta gráfica preferida para gestionar y visualizar los datos en MongoDB. Dada la exposición del puerto 27017, conectar Compass es sencillo. Por ahora. 
